In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as plt
import plotly.express as px
import json


meals_df = pd.read_csv('german-canteens(filtered)(2).csv', sep='@', encoding='utf8')
columns_to_convert = ['student_price', 'employee_price', 'guest_price']

for column in columns_to_convert:
    meals_df[column] = pd.to_numeric(meals_df[column], errors='coerce')

meals_df['average_price'] = meals_df[columns_to_convert].mean(axis=1)
avg_price_per_mensa = meals_df.groupby('mensa_id')['average_price'].mean().reset_index()

with open('further_updated_german_canteens.json', 'r', encoding='utf-8') as file:
    geo_data = json.load(file)

geo_df = pd.DataFrame(geo_data)
geo_df_filtered = geo_df[geo_df['name'].str.contains("Mensa" or "mensa", case=False, na=False)]
merged_data = pd.merge(avg_price_per_mensa, geo_df_filtered, left_on='mensa_id', right_on='id')
top_10_expensive_mensas = merged_data.nlargest(100, 'average_price')
top_10_affordable_mensas = merged_data.nsmallest(100, 'average_price')
top_10_expensive_mensas['category'] = 'Expensive'
top_10_affordable_mensas['category'] = 'Affordable'
combined_data = pd.concat([top_10_expensive_mensas, top_10_affordable_mensas])

def extract_coordinate(coord, index):
    try:
        return coord[index]
    except TypeError:
        return None

combined_data['latitude'] = combined_data['coordinates'].apply(lambda x: extract_coordinate(x, 0))
combined_data['longitude'] = combined_data['coordinates'].apply(lambda x: extract_coordinate(x, 1))
valid_coords = combined_data.dropna(subset=['latitude', 'longitude'])

fig = px.scatter_geo(valid_coords,
                     lat='latitude',
                     lon='longitude',
                     hover_name='name',
                     color='category',  
                     projection='mercator',
                     title='Top 100 Most Expensive and Affordable Canteens with "Mensa" in Name in Germany',
                     color_discrete_map={'Expensive': 'red', 'Affordable': 'green'}  
                    )

fig.update_traces(marker=dict(size=10))  
fig.update_layout(mapbox_style="open-street-map", geo=dict(resolution=50))
fig.update_geos(fitbounds="locations", visible=True, countrycolor="Black",
                showcountries=True, countrywidth=0.5)

#fig.write_html('combined_mensas_plot_fixed_size.html')
fig.show()
# Write the combined_data DataFrame to a CSV file
combined_data.to_csv('RQ7combined_expensive_motst_least_affordable_mensas.csv', index=False, encoding='utf-8')

fig.write_html('plot1000000.html')

In [ ]:
##############################DIE HIER IST RICHTIG###########################################

import pandas as pd
import json
import plotly.graph_objects as go

data = pd.read_csv('german-canteens(filtered)(2).csv', sep='@', encoding='utf8')
#data = pd.read_csv('RQ7combined_german_state_average_mensas.csv', sep='@', encoding='utf8')
#RQ7combined_german_state_average_mensas.csv
data2 = pd.read_json('further_updated_german_canteens.json', encoding='utf8')

with open('deutschland_updated.geo.json', 'r', encoding='utf8') as file:
    german_states_geo = json.load(file)

data3 = data.join(data2.set_index('id'), on='mensa_id')

#data3.drop(columns=data3.columns[0:2], axis=1, inplace=True)
data3['student_price'] = pd.to_numeric(data3['student_price'], errors='coerce')
data3['employee_price'] = pd.to_numeric(data3['employee_price'], errors='coerce')
data3['guest_price'] = pd.to_numeric(data3['guest_price'], errors='coerce')

avg_student_prices = data3.groupby('state-id')['student_price'].mean().reset_index()
avg_employee_prices = data3.groupby('state-id')['employee_price'].mean().reset_index()
avg_guest_prices = data3.groupby('state-id')['guest_price'].mean().reset_index()

range_color_student = (avg_student_prices['student_price'].min(), min(avg_student_prices['student_price'].max(), 7))
range_color_employee = (avg_employee_prices['employee_price'].min(), min(avg_employee_prices['employee_price'].max(), 7))
range_color_guest = (avg_guest_prices['guest_price'].min(), min(avg_guest_prices['guest_price'].max(), 7))

fig = go.Figure()
colorscale = "Viridis"

fig.add_trace(go.Choroplethmapbox(
    geojson=german_states_geo,
    locations=avg_student_prices['state-id'],
    z=avg_student_prices['student_price'],
    colorscale=colorscale,
    zmin=range_color_student[0],
    zmax=5,  
    name='Average Student Prices',
    visible=True
))

fig.add_trace(go.Choroplethmapbox(
    geojson=german_states_geo,
    locations=avg_employee_prices['state-id'],
    z=avg_employee_prices['employee_price'],
    colorscale=colorscale,
    zmin=range_color_employee[0],
    zmax=7,  
    name='Average Employee Prices',
    visible=False
))

fig.add_trace(go.Choroplethmapbox(
    geojson=german_states_geo,
    locations=avg_guest_prices['state-id'],
    z=avg_guest_prices['guest_price'],
    colorscale=colorscale,
    zmin=range_color_guest[0],
    zmax=10,  
    name='Average Guest Prices',
    visible=False
))

fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=5,
    mapbox_center={"lat": 51, "lon": 10},
    title='Average Price Distribution in German Canteens',
    updatemenus=[{
        'buttons': [
            {'label': 'Average Student Prices', 'method': 'update', 'args': [{'visible': [True, False, False]}]},
            {'label': 'Average Employee Prices', 'method': 'update', 'args': [{'visible': [False, True, False]}]},
            {'label': 'Average Guest Prices', 'method': 'update', 'args': [{'visible': [False, False, True]}]}
        ],
        'direction': 'down',
        'showactive': True,
    }]
)

fig.show()
combined_data.to_csv('RQ7combined_german_state_average_mensas.csv', index=False, encoding='utf-8')